In [ ]:
!pip install rarfile

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-9mcmllnj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-9mcmllnj
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using c

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import math

import cv2
from sklearn import metrics

import pickle
import clip
import torch
from PIL import Image
from tqdm.notebook import tqdm
import torch.nn.functional as F
import io
import rarfile


# mounting drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# # loading pre-trained CLIP model
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/16", device=device)

100%|███████████████████████████████████████| 335M/335M [00:06<00:00, 53.8MiB/s]


In [ ]:
# reading annotations file
file_path = "/content/drive/MyDrive/Assignments/emotion_recognition_clip/DATA_FOR_CLIP/"
# file_path = "/content/drive/MyDrive/DATA_FOR_CLIP/"
annotations = pd.read_pickle(os.path.join(file_path, 'test-annotation-e',
                                          'annotation_test.pkl'))

classify_labels = list(annotations.keys()) # capturing output classes
ground_truth = pd.DataFrame(annotations)

# loading pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)
text = clip.tokenize(classify_labels).to(device)
text_features = model.encode_text(text)

100%|████████████████████████████████████████| 335M/335M [00:02<00:00, 118MiB/s]


In [ ]:
skip_feature_extraction = True
zip_data_path = "/content/drive/MyDrive/Assignments/emotion_recognition_clip/dataset_zips/test_imagesv2.rar" # location of test dataset frames
rf = rarfile.RarFile(zip_data_path)
image_list_zip = rf.infolist()
image_index = [x.filename for x in image_list_zip if x.filename.endswith('jpg')]
image_names = [x[12:] for x in image_index]

feature_path = '/content/drive/MyDrive/Assignments/emotion_recognition_clip/dataset_zips/base-v16-test'


if not skip_feature_extraction:
  batch_size = 100
  all_prob = []
  for batch_num in tqdm(range(len(image_index)//batch_size)):
      images = []
      for video_name in tqdm(image_list_zip[(batch_num)*batch_size:(batch_num+1)*batch_size]):
          if '.jpg' in video_name.filename:
              img_bytes =rf.read(video_name)        # 1
              img_data = Image.open(io.BytesIO(img_bytes))
              image = preprocess(img_data).unsqueeze(0).to(device)
              images.append(image)
      images_batch = torch.cat(images, dim=0)
      # Get image features from the CLIP model
      with torch.no_grad():

          image_features = model.encode_image(images_batch)
          torch.save(image_features, os.path.join(feature_path,'feat_image_batch_{}.pt'.format(batch_num)))
          text_features = model.encode_text(text)
          logits = 100.0 * image_features @ text_features.T
          probs_all = logits.softmax(dim=1)
          all_prob.append(probs_all)
      del image_features
      del images_batch
      torch.cuda.empty_cache()
else:
  os.chdir(feature_path)
  feature_files = glob.glob("*.pt")
  load_tensors = [torch.load(file_name) for file_name in feature_files]
  all_feat = torch.concat(load_tensors)
  logits = (all_feat/all_feat.norm(dim=1, keepdim=True) )@ (text_features/text_features.norm(dim=1, keepdim=True)).T


In [ ]:
ground_truth = pd.DataFrame(annotations).reset_index()
ground_truth['image_name'] = ground_truth['index'].map(lambda x: '.'.join(x.split('.')[:-1]))
ground_truth['image_name'] = ground_truth['image_name'].map(lambda x: x.replace('_',''))

# ground_truth['frame_name'] = ground_truth['index'].map(lambda x: '.'.join(x.split('.')[-1:]))
ground_truth = ground_truth.drop(['index'], axis = 1)

# ground_truth = ground_truth.loc[ground_truth.image_name.isin(predictions.index)]
ground_truth = ground_truth.set_index(['image_name'])

ground_truth.head()

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness
image_name,,,,,,
htH89DBizno.004,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222
pwf-KszNlk.001,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556
MuYYY3XaJ7Q.001,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222
0MB91ku0eEw.005,0.411215,0.458333,0.714286,0.660194,0.570093,0.400000
WpEZOSrENL0.003,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111


In [ ]:
def metrics_calc(pred, act):
    print ('*'*100)
    print('Evaluating mean accuracy on test data \n')
    acc = []
    # calculating mean accuracy for each trait
    for column in pred:
        # print (column)
        # acc_class = 1 - ((abs(pred[column] - act[column])).mean()/abs(act[column] - act[column].mean()).sum())
        acc_class = 1 - ((abs(pred[column] - act[column])).mean())
        print ('accuracy for {} is {}'.format(column, acc_class))
        acc.append(acc_class)

    m_accuracy = np.mean(acc)
    print ('mean accuracy over all traits is {}'.format( m_accuracy))


In [ ]:
# predictions = pd.DataFrame(logits.detach().numpy(), index = image_names, columns = classify_labels)

# for col in predictions:
#   predictions[col] = (predictions[col] - predictions[col].min())/ (predictions[col].max() - predictions[col].min() )

# predictions = predictions.reset_index()
# predictions['image_name'] = predictions['index'].map(lambda x: ''.join(x.split('_')[:-2])) # removing frame number to get only the image name

# predictions = predictions.drop(['index'], axis = 1)
# predictions = predictions.groupby(['image_name']).mean() # average score at image level (grouping all frames result into single result)
# predictions.head()

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness
image_name,,,,,,
--Ymqszjv54.000,0.729487,0.732367,0.604590,0.710766,0.493054,0.735813
-10-QQDOME.001,0.633748,0.686190,0.538523,0.623684,0.541860,0.630134
-10-QQDOME.002,0.641416,0.692831,0.564087,0.631459,0.556379,0.661538
-10-QQDOME.005,0.636987,0.683582,0.544682,0.616827,0.542376,0.643735
-4J4xkfN5cI.000,0.536543,0.518775,0.494888,0.448353,0.495909,0.435092


In [ ]:
predictions = pd.DataFrame()

for class_label in classify_labels:
  labels_new= ['A Person with '+class_label +' impression' , "A Person with neutral impression"]
  print (labels_new)
  text = clip.tokenize(labels_new).to(device)
  text_features = model.encode_text(text)
  logits = (all_feat/all_feat.norm(dim=1, keepdim=True) )@ (text_features/text_features.norm(dim=1, keepdim=True)).T
  probs = logits.softmax(dim=-1).detach().numpy()
  predictions[class_label] = pd.DataFrame(probs, index = image_names, columns = [class_label , "others"])[class_label]


# for col in predictions:
#   predictions[col] = (predictions[col] - predictions[col].min())/ (predictions[col].max() - predictions[col].min() )

predictions = predictions.reset_index()
predictions['image_name'] = predictions['index'].map(lambda x: ''.join(x.split('_')[:-2])) # removing frame number to get only the image name

predictions = predictions.drop(['index'], axis = 1)
predictions = predictions.groupby(['image_name']).mean() # average score at image level (grouping all frames result into single result)
metrics_calc(predictions.sort_index(), ground_truth.sort_index())

['A Person with extraversion impression', 'A Person with neutral impression']
['A Person with neuroticism impression', 'A Person with neutral impression']
['A Person with agreeableness impression', 'A Person with neutral impression']
['A Person with conscientiousness impression', 'A Person with neutral impression']
['A Person with interview impression', 'A Person with neutral impression']
['A Person with openness impression', 'A Person with neutral impression']
****************************************************************************************************
Evaluating mean accuracy on test data 

accuracy for extraversion is 0.877426106296967
accuracy for neuroticism is 0.8750580517947674
accuracy for agreeableness is 0.8830352047875032
accuracy for conscientiousness is 0.8724541233744725
accuracy for interview is 0.8815556547502491
accuracy for openness is 0.8692274704731173
mean accuracy over all traits is 0.8764594352461795
